## HRA API Usage
This notebook shows how to use the HRA API method calls provided by the `hra_api_client` package. We will show how to set up the client and make a few calls. Refer to the usage notebook [here](https://github.com/x-atlas-consortia/hra-api/blob/main/notebooks/hra-api-client-usage.ipynb) and to the API documentation on <https://apps.humanatlas.io/api/> for additional documentation, methods, and examples.

### Install package
For this notebook, we'll install the `hra-api-client` and a set of HRA UI widgets called `hra-jupyter-widgets`.

In [1]:
%pip install hra-api-client hra-jupyter-widgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


<a id='imports'></a>
### Imports 

We import the hra-api-client as follows:

In [2]:
import hra_api_client

Although we can use hra-api-client for all our tasks, it is easier if we have individual imports as follows:

In [3]:
from hra_api_client.api import v1_api
from hra_api_client.api import hra_pop_api
from hra_api_client.api import ds_graph_api
from hra_api_client.models.aggregate_count import AggregateCount
from hra_api_client.models.database_status import DatabaseStatus
from hra_api_client.models.error_message import ErrorMessage
from hra_api_client.models.flat_spatial_placement import FlatSpatialPlacement
from hra_api_client.models.get_spatial_placement_request import GetSpatialPlacementRequest
from hra_api_client.models.min_max import MinMax
from hra_api_client.models.ontology_tree import OntologyTree
from hra_api_client.models.sparql_query_request import SparqlQueryRequest
from hra_api_client.models.spatial_entity import SpatialEntity
from hra_api_client.models.spatial_scene_node import SpatialSceneNode
from hra_api_client.models.spatial_search import SpatialSearch
from hra_api_client.models.tissue_block import TissueBlock

For the purposes of this demonstration we need some other libraries imported here: 

In [4]:
import time
from pprint import pprint
from hra_jupyter_widgets import ModelViewer
import json

### Creating the API Instance 
We will now create an instance of the API client to use methods provided by the HRA API.

In [5]:
# Client configuration, the default API endpoint is https://apps.humanatlas.io/api
configuration = hra_api_client.Configuration(
    host = "https://apps.humanatlas.io/api" 
)

# Base Client Object
api_client = hra_api_client.ApiClient(configuration)

# Instance with which to call the primary /v1 routes from our API
api_instance = v1_api.V1Api(api_client)

# Instance with which to call the /hra-pop routes
hra_pop_api_instance = hra_pop_api.HraPopApi(api_client)

# Instance with which to call the /ds-graph routes
ds_graph_api_instance = ds_graph_api.DsGraphApi(api_client)

## Aggregate statistics
To get a quick idea of what data is available in the HRA API, you can use the `aggregate_results` method. With no options, it returns various counts for all default (federated) data.

In [6]:
api_response = api_instance.aggregate_results()
for x in api_response:
  print(x.count, x.label)

18 Consortia
49 Tissue Data Providers
724 Donors
30 Organs
2135 Extraction Sites
2638 Tissue Blocks
2212 Tissue Sections
8045 Tissue Datasets


## Get a session token for HuBMAP data
You can limit to certain sets of data by creating a session token. We will create a session for just HuBMAP data. Filters, discussed later, can also be used here to make a smaller set of data for more targeted studies.


In [7]:
# Session configuration. You can add your HuBMAP token after ?token= to get private data
session_configuration = { "dataSources": ["https://apps.humanatlas.io/api/ds-graph/hubmap?token="] }

# Get a session token for this configuration
api_response = api_instance.session_token(session_configuration)
token=api_response.token
print(token)

ad224e52857dde02cbb05a10bfb807c7


After we get the session token, we wait to make sure the session is ready.

In [8]:
db_ready = False
while not db_ready:
    api_response = api_instance.db_status(token)
    print(api_response)
    if api_response.status == 'Ready':
        db_ready = True
    else:
        print('Database not ready yet! Retrying...')
        time.sleep(2)

status='Ready' checkback=3600000 load_time=11388 message='Dataset ready'


## Examine the HuBMAP data
Now we can check the data available in the session we created

In [9]:
# get aggregate statistics
api_response = api_instance.aggregate_results(token=token)
for x in api_response:
  print(x.count, x.label)

1 Consortia
12 Tissue Data Providers
189 Donors
23 Organs
814 Extraction Sites
834 Tissue Blocks
1259 Tissue Sections
4975 Tissue Datasets


Let's get all of the tissue blocks (with a RUI location) in HuBMAP

In [10]:
# get all tissue blocks
api_response = api_instance.tissue_blocks(token=token)

# print first five block links
for x in api_response[0:5]:
  print(x.link)

https://portal.hubmapconsortium.org/browse/sample/00149d1450ae558e2354936b13045f1d
https://portal.hubmapconsortium.org/browse/sample/00bf92f03e3eadf91552850d32eff9e4
https://portal.hubmapconsortium.org/browse/sample/0101a0807f72d7028231205d5062910a
https://portal.hubmapconsortium.org/browse/sample/0124d846cc36c40d3ce9b40cc697875f
https://portal.hubmapconsortium.org/browse/sample/015f9976f3fed4c35dbbaad622a413d9


## Filter the HuBMAP data

Certain parameters can be used to filter the data. Most methods that query the data have these filters to use:
<pre>
    age - (MinMax) (optional) (min = 1.0; max = 110.0)
    bmi - (MinMax) (optional) (min = 13.0; max = 83.0)
    sex - (string)  (optional) (male, female, both)
    technologies - (string) (optional) (List of technologies - api_instance.technology_names)
    providers - (string) (optional) 
    ontology_terms - (string) (optional) (List of ontology terms)
    cell_type_terms - (string) (optional) (List of ontology tems)
    spatial - SpatialSearch(x, y, z, radius, target) (optional) 
    token - (string) (optional) the session token, if not provided it will use the default dataset
</pre>

Example instantantiations of the filters are below.

In [11]:
age = MinMax(min=10.0, max=100.0)
bmi = MinMax(min=0.0, max=40.0)
sex = 'Male'
technologies = []
providers = []
ontology_terms = []
cell_type_terms = []

spatial_obj = SpatialSearch(
    x=38.0,
    y=53.0,
    z=41.0,
    radius=44.0,
    target='https://purl.humanatlas.io/ref-organ/kidney-male-left/v1.3#primary'
)

api_response = api_instance.aggregate_results(
    age=age.to_json(),
    ontology_terms=ontology_terms,
    cell_type_terms=cell_type_terms,
    providers=providers,
    sex=sex,
    spatial=json.dumps([spatial_obj.to_dict()]),        
    technologies=technologies,
    token=token
)

for x in api_response:
    print(x.count, x.label)

1 Consortia
3 Tissue Data Providers
20 Donors
1 Organs
27 Extraction Sites
34 Tissue Blocks
117 Tissue Sections
220 Tissue Datasets


Another way to manage these filters is to create a Dictionary and use python's `**kwargs` syntax to create and pass around a filter object. We'll use this filter object for several methods below.

In [12]:
age = MinMax(min=10.0, max=100.0)

spatial_obj = SpatialSearch(
    x=38.0,
    y=53.0,
    z=41.0,
    radius=44.0,
    target='https://purl.humanatlas.io/ref-organ/kidney-male-left/v1.3#primary'
)

filter = {
  "age" : age.to_json(),
  "sex": "Male",
  "technologies": [],
  "providers": [],
  "ontology_terms": [],
  "cell_type_terms": ["http://purl.obolibrary.org/obo/CL_0000000"],
  "spatial": json.dumps([spatial_obj.to_dict()])
}

# get filtered aggregate statistics
api_response = api_instance.aggregate_results(token=token, **filter)
for x in api_response:
  print(x.count, x.label)
  


1 Consortia
3 Tissue Data Providers
20 Donors
1 Organs
27 Extraction Sites
34 Tissue Blocks
117 Tissue Sections
220 Tissue Datasets


In [13]:
# get anatomical structures that the tissue blocks collided with
api_response = api_instance.ontology_term_occurences(token=token, **filter)

# Print the first 5 results
for (anatomical_structure, count) in list(api_response.items())[0:5]:
  print(anatomical_structure, count)

http://purl.obolibrary.org/obo/UBERON_0000362 17
http://purl.obolibrary.org/obo/UBERON_0001224 1
http://purl.obolibrary.org/obo/UBERON_0001225 18
http://purl.obolibrary.org/obo/UBERON_0001226 1
http://purl.obolibrary.org/obo/UBERON_0001227 3


In [14]:
# get cell types located in anatomical structures that the tissue blocks collided with
api_response = api_instance.cell_type_term_occurences(token=token, **filter)

# Print the first 5 results
for (cell_type, count) in list(api_response.items())[0:5]:
  print(cell_type, count)

http://purl.obolibrary.org/obo/CL_0000000 34
https://purl.org/ccf/ASCTB-TEMP_capsule-mesenchymal-stromal-cell 18


## Query the HRA Knowledge Graph with SPARQL
The HRA API provides a SPARQL endpoint to query the HRA knowledge graph. This allows you to select out very specific data using advanced SPARQL querying. 

To demonstrate, let's get all the anatomical structures in the latest HRA release.

In [15]:
query = '''
PREFIX ccf: <http://purl.org/ccf/>
SELECT ?as ?label
FROM <https://purl.humanatlas.io/collection/hra>
WHERE {
  ?as ccf:ccf_asctb_type ?type ;
    ccf:ccf_pref_label ?label .
  FILTER(?type = 'AS')
}
'''
api_response = api_instance.sparql(query=query)
rows = api_response['results']['bindings']

as_lookup = dict([ (row['as']['value'], row['label']['value']) for row in rows ])

# Print the first 5 results
for (anatomical_structure, label) in list(as_lookup.items())[0:5]:
  print(anatomical_structure, label)

http://purl.obolibrary.org/obo/UBERON_0000006 Islet of Langerhans
http://purl.obolibrary.org/obo/UBERON_0000016 Endocrine pancreas
http://purl.obolibrary.org/obo/UBERON_0000017 Exocrine pancreas
http://purl.obolibrary.org/obo/UBERON_0000061 anatomical structure
http://purl.obolibrary.org/obo/UBERON_0000362 renal medulla


If your SPARQL query is longer (thousands of characters), you can do a SPARQL POST which allows for sending more data.

In [16]:
api_response = api_instance.sparql_post({'query': query})
rows = api_response['results']['bindings']

as_lookup = dict([ (row['as']['value'], row['label']['value']) for row in rows ])

# Print the first 5 results
for (anatomical_structure, label) in list(as_lookup.items())[0:5]:
  print(anatomical_structure, label)

http://purl.obolibrary.org/obo/UBERON_0000006 Islet of Langerhans
http://purl.obolibrary.org/obo/UBERON_0000016 Endocrine pancreas
http://purl.obolibrary.org/obo/UBERON_0000017 Exocrine pancreas
http://purl.obolibrary.org/obo/UBERON_0000061 anatomical structure
http://purl.obolibrary.org/obo/UBERON_0000362 renal medulla


## Working with extraction sites (aka RUI locations)
After obtaining an extraction site either via the RUI or from queries like above, there are a few things you can do with this data.

### Generate a 3D corridor

We can compute a corridor from any extraction site. A corridor shows where else an extraction site could go in the same organ and still maintain the same overlaps with anatomical structures. We will take an example extraction site and see what it's corridor looks like.

An example extraction site:

In [17]:
# Provide a RUI location via the RUI at https://apps.humanatlas.io/rui/
extraction_site = {
    "@context": "https://hubmapconsortium.github.io/ccf-ontology/ccf-context.jsonld",
    "@id": "http://purl.org/ccf/1.5/cc837df1-2788-4510-9c21-9eb0e9b310a5",
    "@type": "SpatialEntity",
    "creator": "demo demo",
    "creator_first_name": "demo",
    "creator_last_name": "demo",
    "creation_date": "2024-08-20",
    "ccf_annotations": [
        "http://purl.obolibrary.org/obo/UBERON_0002015",
        "http://purl.obolibrary.org/obo/UBERON_0002189"
    ],
    "x_dimension": 10,
    "y_dimension": 10,
    "z_dimension": 10,
    "dimension_units": "millimeter",
    "placement": {
        "@context": "https://hubmapconsortium.github.io/ccf-ontology/ccf-context.jsonld",
        "@id": "http://purl.org/ccf/1.5/cc837df1-2788-4510-9c21-9eb0e9b310a5_placement",
        "@type": "SpatialPlacement",
        "target": "https://purl.humanatlas.io/ref-organ/kidney-female-right/v1.3#primary",
        "placement_date": "2024-08-20",
        "x_scaling": 1,
        "y_scaling": 1,
        "z_scaling": 1,
        "scaling_units": "ratio",
        "x_rotation": 0,
        "y_rotation": 0,
        "z_rotation": 0,
        "rotation_order": "XYZ",
        "rotation_units": "degree",
        "x_translation": 72.366,
        "y_translation": 79.067,
        "z_translation": 34.77,
        "translation_units": "millimeter"
    }
}

In [18]:
# Generate a corridor for the extraction site
model = api_instance.corridor(extraction_site)

# Display it in 3D
viewer = ModelViewer(data=model)
display(viewer)

### Find anatomical structure collisions for an extraction site
For any extraction site, we can analyze what anatomical structures it collides with in the 3D reference organ and by how much.

In [19]:
# Get mesh-based collisions
collisions = api_instance.collisions(extraction_site)

pprint(collisions[0:5])

[{'AS_volume': 10055.261678806379,
  'id': 'https://purl.humanatlas.io/ref-organ/kidney-female-right/v1.3#VH_F_kidney_capsule_R',
  'intersection_volume': 87,
  'is_closed': True,
  'label': 'kidney capsule',
  'node_name': 'VH_F_kidney_capsule_R',
  'organ': 'kidney-female-right',
  'percentage_of_AS': 0.008652186564509919,
  'percentage_of_tissue_block': 0.087,
  'representation_of': 'http://purl.obolibrary.org/obo/UBERON_0002015',
  'tissue_volume': 1000},
 {'AS_volume': 84566.4834520329,
  'id': 'https://purl.humanatlas.io/ref-organ/kidney-female-right/v1.3#VH_F_outer_cortex_of_kidney_R',
  'intersection_volume': 366,
  'is_closed': True,
  'label': 'outer cortex of kidney',
  'node_name': 'VH_F_outer_cortex_of_kidney_R',
  'organ': 'kidney-female-right',
  'percentage_of_AS': 0.004327955769942823,
  'percentage_of_tissue_block': 0.366,
  'representation_of': 'http://purl.obolibrary.org/obo/UBERON_0002189',
  'tissue_volume': 1000}]


## Conclusion
This concludes this tutorial. Refer to the usage notebook [here](https://github.com/x-atlas-consortia/hra-api/blob/main/notebooks/hra-api-client-usage.ipynb) and to the API documentation on <https://apps.humanatlas.io/api/> for additional documentation, methods, and examples. If you have any issues, problems, or questions, feel free to file an issue on the HRA API GitHub at <https://github.com/x-atlas-consortia/hra-api>.